In [1]:
import os, sys
sys.path.append(os.path.dirname("/workspace/CogAgent/basic_demo"))
from PIL import Image
from functools import partial
import torch
import time
import argparse
import numpy as np
from sat.mpu import get_model_parallel_world_size
from sat.model import AutoModel
from utils.utils import chat, llama2_tokenizer, llama2_text_processor, get_image_processor, parse_response, get_grounding_image_processor
from utils.models import CogAgentModel, CogVLMModel
from utils.utils import TestItemDataset as ItemDataset
from sat.training.deepspeed_training import inference_main
rank = int(os.environ.get('RANK', 0))
world_size = int(os.environ.get('WORLD_SIZE', 1))

print("Rank: ", rank)
print("World size: ", world_size)

[2024-03-25 00:07:08,179] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
Rank:  0
World size:  1


In [2]:
from custom_demo import data_collator, load_model, create_dataset_function, forward_step

In [3]:
from argparse import Namespace
from utils.utils import llama2_tokenizer

# Define your arguments
args = Namespace(
    from_pretrained="../finetune_demo/checkpoints/finetune-cogagent-vqa-03-21-19-37/",
    local_tokenizer="lmsys/vicuna-7b-v1.5",
    max_length=1024,
    top_p=0.4,
    top_k=1,
    temperature=0.8,
    version="chat_old",
    quant=None,
    fp16=True,
    bf16=False,
    stream_chat=False,
    gnd_image_pix=512,
    use_lora=True
)

vg_token = "给"
tokenizer = llama2_tokenizer(args.local_tokenizer, signal_type=args.version)
args.vg_token_idx = tokenizer.convert_tokens_to_ids(vg_token)

print("Total number of tokens: ", tokenizer.vocab_size)
print("Using VG token: ", vg_token, " with index: ", args.vg_token_idx)
print("\n\nargs:", args)

Total number of tokens:  32000
Using VG token:  给  with index:  31999


args: Namespace(from_pretrained='../finetune_demo/checkpoints/finetune-cogagent-vqa-03-21-19-37/', local_tokenizer='lmsys/vicuna-7b-v1.5', max_length=1024, top_p=0.4, top_k=1, temperature=0.8, version='chat_old', quant=None, fp16=True, bf16=False, stream_chat=False, gnd_image_pix=512, use_lora=True, vg_token_idx=31999)


In [4]:
model, image_processor, cross_image_processor, text_processor_infer, grounding_image_processor = load_model(args)
print("Model Ready For Inference", flush=True)
print("Model Size: ", sum(p.numel() for p in model.parameters())/1e6, "M", flush=True)

[2024-03-25 00:07:14,571] [INFO] building FineTuneTrainCogAgentModelNew model ...
[2024-03-25 00:07:14,575] [INFO] [RANK 0] > initializing model parallel with size 1
[2024-03-25 00:07:14,576] [INFO] [RANK 0] You didn't pass in LOCAL_WORLD_SIZE environment variable. We use the guessed LOCAL_WORLD_SIZE=1. If this is wrong, please pass the LOCAL_WORLD_SIZE manually.
[2024-03-25 00:07:14,577] [INFO] [RANK 0] You are using model-only mode.
For torch.distributed users or loading model parallel models, set environment variables RANK, WORLD_SIZE and LOCAL_RANK.
/usr/local/lib/python3.10/dist-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3549.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]



Loaded pretrained groundindino model from ../groundingdino_swinb_cogcoor.pth with msg: _IncompatibleKeys(missing_keys=['transformer.tgt_embed.weight'], unexpected_keys=['label_enc.weight', 'bert.embeddings.position_ids', 'bert.embeddings.word_embeddings.weight', 'bert.embeddings.position_embeddings.weight', 'bert.embeddings.token_type_embeddings.weight', 'bert.embeddings.LayerNorm.weight', 'bert.embeddings.LayerNorm.bias', 'bert.encoder.layer.0.attention.self.query.weight', 'bert.encoder.layer.0.attention.self.query.bias', 'bert.encoder.layer.0.attention.self.key.weight', 'bert.encoder.layer.0.attention.self.key.bias', 'bert.encoder.layer.0.attention.self.value.weight', 'bert.encoder.layer.0.attention.self.value.bias', 'bert.encoder.layer.0.attention.output.dense.weight', 'bert.encoder.layer.0.attention.output.dense.bias', 'bert.encoder.layer.0.attention.output.LayerNorm.weight', 'bert.encoder.layer.0.attention.output.LayerNorm.bias', 'bert.encoder.layer.0.intermediate.dense.weight', 

[2024-03-25 00:08:52,486] [INFO] [RANK 0]  > number of parameters on model parallel rank 0: 18432088128
INFO:sat:[RANK 0]  > number of parameters on model parallel rank 0: 18432088128
[2024-03-25 00:09:59,173] [INFO] [RANK 0] 

Adding LoRA to the model for inference. Expecting the model weights to be in the LoRA format. If not, please stop the process and fix it
\m
INFO:sat:[RANK 0] 

Adding LoRA to the model for inference. Expecting the model weights to be in the LoRA format. If not, please stop the process and fix it
\m
[2024-03-25 00:09:59,178] [INFO] [RANK 0] replacing layer 0 attention with lora
INFO:sat:[RANK 0] replacing layer 0 attention with lora
[2024-03-25 00:10:00,213] [INFO] [RANK 0] replacing layer 0 cross attention with lora
INFO:sat:[RANK 0] replacing layer 0 cross attention with lora
[2024-03-25 00:10:00,971] [INFO] [RANK 0] replacing layer 1 attention with lora
INFO:sat:[RANK 0] replacing layer 1 attention with lora
[2024-03-25 00:10:02,108] [INFO] [RANK 0] replacing 

Model Ready For Inference
Model Size:  18552.612928 M


In [5]:
print(model)

FineTuneTrainCogAgentModelNew(
  (mixins): ModuleDict(
    (lm): LMMixin(
      (lm_head): ColumnParallelLinear()
    )
    (eva): ImageMixin(
      (vit_model): EVA2CLIPModel(
        (mixins): ModuleDict(
          (patch_embedding): ImagePatchEmbeddingMixin(
            (proj): Conv2d(3, 1792, kernel_size=(14, 14), stride=(14, 14))
          )
          (pos_embedding): InterpolatedPositionEmbeddingMixin()
          (final): IdentityMixin()
          (newpost): NewLayerForward()
          (xattn): XAttn()
          (lora): LoraMixin()
        )
        (transformer): BaseTransformer(
          (embedding_dropout): Dropout(p=0.1, inplace=False)
          (word_embeddings): Embedding(1, 1792)
          (position_embeddings): Embedding(257, 1792)
          (layers): ModuleList(
            (0-62): 63 x BaseTransformerLayer(
              (input_layernorm): LayerNorm(torch.Size([1792]), eps=1e-06, elementwise_affine=True)
              (attention): SelfAttention(
                (query_

In [6]:
dataset_valid = create_dataset_function(image_processor, text_processor_infer, cross_image_processor, grounding_image_processor, vg_token, path="../test_data/apollo_ferret_noscale.json", args=args)
data_loader = torch.utils.data.DataLoader(dataset_valid, batch_size=1, collate_fn=partial(data_collator, cross_image_processor=cross_image_processor))

[2024-03-25 00:14:33,963] [INFO] [RANK 0] find 1 samples in all...
INFO:sat:[RANK 0] find 1 samples in all...


all_data: {'Question': 'Task: Give me the list of all product manager with minimum 5 years of experience working in a series A company with minimum 1M in revenue. \n Previous Action: None \nGive me the next action?', 'Answer': 'This is a dummy answer and a dummy [0.179,0.452,0.385,0.51] button', 'workflow': 'anurag-wf-10_augmented_cropped', 'imagePath': '/workspace/CogAgent/test_data//anish-wf-6_a.png'}
Creating dataset using the VG token:  给


In [8]:
lm_logits, bbox_outputs = inference_main(args, model=model, forward_step_function=forward_step, data_loader=data_loader)
print("Inference Done", flush=True)

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:90: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Inference Done


In [10]:
print("bbox_outputs: ", bbox_outputs['pred_boxes'])

bbox_outputs:  tensor([[[0.0826, 0.1047, 0.1525, 0.0331],
         [0.0779, 0.1162, 0.1530, 0.0384],
         [0.0924, 0.3704, 0.1726, 0.0319],
         ...,
         [0.3192, 0.2665, 0.2923, 0.0349],
         [0.2221, 0.3241, 0.1146, 0.0330],
         [0.2731, 0.4535, 0.1855, 0.0465]]], device='cuda:0')


In [9]:
print("lm_logits: ", lm_logits)

lm_logits:  tensor([[[ 1.0371, -0.1876,  0.1923,  ...,  1.2939,  2.1895,  1.4922],
         [ 3.2578,  3.2871, -0.4783,  ...,  2.9355,  3.9180,  2.9355],
         [-2.6777, -6.6641, -1.6504,  ..., -0.7275, -2.1895, -0.1832],
         ...,
         [-4.7656,  0.6045, 12.1094,  ..., -0.8701, -4.4258, -0.8740],
         [-4.7500,  0.6826, 12.1328,  ..., -0.8037, -4.3867, -0.8442],
         [-4.7383,  0.7798, 12.2422,  ..., -0.7568, -4.3555, -0.8159]]],
       device='cuda:0')


In [14]:
# Now decode the logits to text using the tokenizer
lm_logits_copy = lm_logits.clone()
lm_logits_copy = lm_logits_copy.squeeze(0)
lm_logits_copy = lm_logits_copy.cpu().numpy()
lm_logits_copy = lm_logits_copy[:, 1:]
lm_logits_copy = lm_logits_copy.argmax(axis=-1)
lm_logits_copy = lm_logits_copy.tolist()
print("lm_logits: ", lm_logits_copy)
response = tokenizer.decode(lm_logits_copy)
print("Response: ", response)

lm_logits:  [23195, 24365, 5496, 285, 659, 29870, 13607, 29870, 29870, 5555, 1721, 29994, 659, 2739, 5066, 3814, 29994, 5066, 398, 1919, 29888, 1018, 381, 4720, 12713, 2739, 29888, 29994, 13607, 29994, 381, 29994, 29994, 1550, 7568, 2760, 29994, 23767, 659, 372, 29994, 1, 29994, 29994, 1, 29994, 29899, 29899, 1, 1, 2190, 1856, 1856, 727, 2626, 1856, 1, 1, 4875, 382, 1856, 1721, 1721, 2626, 11596, 12713, 23767, 7568, 659, 7568, 4817, 27758, 1, 1721, 1, 2626, 12713, 1, 1, 4875, 1, 1, 974, 1856, 1721, 23767, 4817, 4817, 1721, 1721, 1, 1, 1, 1721, 1, 12713, 321, 2626, 7568, 2626, 1431, 7568, 1856, 1721, 1, 1, 1, 1, 4875, 1, 1431, 1, 727, 292, 29955, 1, 2626, 23767, 7568, 382, 2626, 1431, 29994, 730, 1, 1, 1, 1, 382, 12713, 9075, 1856, 292, 7568, 1431, 2701, 2626, 3289, 2626, 730, 1, 382, 382, 382, 1, 1, 7568, 7568, 292, 896, 2190, 382, 29994, 730, 382, 382, 382, 382, 382, 1721, 11596, 29888, 321, 1912, 7932, 5869, 730, 1721, 3289, 896, 1633, 4720, 29994, 382, 382, 382, 382, 292, 1550, 318,

In [16]:
# Now decode the logits to text using the tokenizer
import torch.nn.functional as F
lm_logits_copy = lm_logits.clone()
lm_logits_copy = F.softmax(lm_logits_copy, dim=-1)
lm_logits_copy = lm_logits_copy.squeeze(0)
lm_logits_copy = lm_logits_copy.cpu().numpy()
lm_logits_copy = lm_logits_copy[:, 1:]
lm_logits_copy = lm_logits_copy.argmax(axis=-1)
lm_logits_copy = lm_logits_copy.tolist()
print("lm_logits: ", lm_logits_copy)
response = tokenizer.decode(lm_logits_copy)
print("Response: ", response)

lm_logits:  [23195, 24365, 5496, 285, 659, 29870, 13607, 29870, 29870, 5555, 1721, 29994, 659, 2739, 5066, 3814, 29994, 5066, 398, 1919, 29888, 1018, 381, 4720, 12713, 2739, 29888, 29994, 13607, 29994, 381, 29994, 29994, 1550, 7568, 2760, 29994, 23767, 659, 372, 29994, 1, 29994, 29994, 1, 29994, 29899, 29899, 1, 1, 2190, 1856, 1856, 727, 2626, 1856, 1, 1, 4875, 382, 1856, 1721, 1721, 2626, 11596, 12713, 23767, 7568, 659, 7568, 4817, 27758, 1, 1721, 1, 2626, 12713, 1, 1, 4875, 1, 1, 974, 1856, 1721, 23767, 4817, 4817, 1721, 1721, 1, 1, 1, 1721, 1, 12713, 321, 2626, 7568, 2626, 1431, 7568, 1856, 1721, 1, 1, 1, 1, 4875, 1, 1431, 1, 727, 292, 29955, 1, 2626, 23767, 7568, 382, 2626, 1431, 29994, 730, 1, 1, 1, 1, 382, 12713, 9075, 1856, 292, 7568, 1431, 2701, 2626, 3289, 2626, 730, 1, 382, 382, 382, 1, 1, 7568, 7568, 292, 896, 2190, 382, 29994, 730, 382, 382, 382, 382, 382, 1721, 11596, 29888, 321, 1912, 7932, 5869, 730, 1721, 3289, 896, 1633, 4720, 29994, 382, 382, 382, 382, 292, 1550, 318,

In [43]:
import torch
import torch.nn.functional as F
def top_k_top_p_filtering(
    logits,
    top_k: int = 0,
    top_p: float = 1.0,
    filter_value: float = -float("Inf"),
    min_tokens_to_keep: int = 1,
    ):
    """Filter a distribution of logits using top-k and/or nucleus (top-p) filtering
    Source: https://huggingface.co/transformers/v3.2.0/_modules/transformers/generation_utils.html
    """
    if top_k > 0:
        top_k = min(max(top_k, min_tokens_to_keep), logits.size(-1))  # Safety check
        # Remove all tokens with a probability less than the last token of the top-k
        indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
        logits[indices_to_remove] = filter_value

    if top_p < 1.0:
        sorted_logits, sorted_indices = torch.sort(logits, descending=True)
        cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

        # Remove tokens with cumulative probability above the threshold (token with 0 are kept)
        sorted_indices_to_remove = cumulative_probs > top_p
        if min_tokens_to_keep > 1:
            # Keep at least min_tokens_to_keep (set to min_tokens_to_keep-1 because we add the first one below)
            sorted_indices_to_remove[..., :min_tokens_to_keep] = 0
        # Shift the indices to the right to keep also the first token above the threshold
        sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
        sorted_indices_to_remove[..., 0] = 0

        # scatter sorted tensors to original indexing
        indices_to_remove = sorted_indices_to_remove.scatter(1, sorted_indices, sorted_indices_to_remove)
        logits[indices_to_remove] = filter_value
    return logits

lm_logits_copy = lm_logits.clone()
print("original logits: ", lm_logits_copy.shape)
filtered_logits = top_k_top_p_filtering(lm_logits_copy[0], top_p=0.9, top_k=1)
probabilities = F.softmax(filtered_logits, dim=-1)
sampled_token = torch.argmax(probabilities, dim=-1)
response = tokenizer.decode(sampled_token[0].tolist())
print("Response: ", response)

Method 1
original logits:  torch.Size([1, 1024, 32000])
Response:  nobody


In [53]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# Your code here
import torch

def beam_search(logits, beam_size):
    # Ensure beam_size is not larger than the size of the last dimension of logits[0, 0]
    assert beam_size <= logits.size(-1), "beam_size must be less than or equal to the size of the last dimension of logits"

    # Initialize the beam with the first token's logits and indices
    beam_scores, beam_indices = torch.topk(logits[0, 0], beam_size)

    # Initialize the sequences with the first tokens
    sequences = beam_indices.tolist()

    for i in range(1, logits.size(1)):
        # Calculate the scores for the next tokens
        scores, indices = torch.topk(logits[0, i], beam_size)

        # Calculate the total scores for the new sequences
        total_scores = beam_scores.view(-1, 1) + scores.view(1, -1)

        # Flatten the total scores and get the top beam_size scores and their indices
        total_scores = total_scores.view(-1)
        top_scores, top_indices = torch.topk(total_scores, beam_size)

        # Get the token indices and sequence indices for the top scores
        next_tokens = indices.view(-1)[top_indices]
        sequence_indices = top_indices // beam_size

        # Convert sequence_indices to a list of integers
        sequences = [[idx] for idx in beam_indices.tolist()]

        # Update the sequences
        sequences = [sequences[j] + [next_tokens[j].item()] for j in sequence_indices]

        # Update the beam scores
        beam_scores = top_scores

    return sequences

lm_logits_copy = lm_logits.clone()
print("original logits: ", lm_logits_copy.shape)
beam_size = 5
sequences = beam_search(lm_logits_copy, beam_size)

# Now decode the sequences to text using the tokenizer
responses = [tokenizer.decode(sequence) for sequence in sequences]

for i, response in enumerate(responses):
    print(f"Response {i+1}: ", response)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
